In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, udf, lit
from pyspark.sql.types import StringType, ArrayType
spark=SparkSession.builder.appName('TryingSpark').getOrCreate()

spark.conf.set("spark.sql.legacy.setCommandRejectsSparkCoreConfs","false") 
spark.conf.set("spark.executor.memory", "3g") 
spark.conf.set("spark.driver.memory", "3g")

# set spark logger level from config file
spark.sparkContext.setLogLevel("WARN")

raw_data = spark.read.json('/Users/lburle/projects/others/twitter-data-pipeline/data/twitter_data_1.json', multiLine=True)

In [ ]:
from nltk.tokenize import sent_tokenize
from nltk import download as nltk_download
from nltk.corpus import stopwords

nltk_download('punkt')
nltk_download('stopwords')

stopwords = stopwords.words('portuguese')

stopwords = spark.sparkContext.broadcast(stopwords)

In [ ]:
df = raw_data.select(explode(raw_data['data']))

In [ ]:
df = df.select('col.id', 'col.created_at', 'col.text', 'col.public_metrics.*')

In [ ]:
@udf(returnType=ArrayType(StringType()))
def _get_tweet_main_words(tweet):
    stopwords_list = stopwords.value
    tokenized_tweets = sent_tokenize(tweet, language='portuguese')
    filtered_tweet = []
    for token in tokenized_tweets:
        for word in token.split():
            if word not in stopwords_list:
                filtered_tweet.append(word)

    return[filtered_tweet]

In [ ]:
df_processed = df.withColumn(
    'main_words',
    _get_tweet_main_words(
        'text'
    )
)

In [ ]:
df_processed.select('text', 'main_words').show(10, truncate=False)